# Lora 


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
DEBUG = False
import os
if DEBUG: 
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
data_path = "content"
import torch

import urllib.request
import zipfile
from pathlib import Path
import pandas as pd
import time, math


In [3]:
from src.dataset_loader import get_enc_dataset
# Set model name
from transformers import GPT2Tokenizer
model_name = "gpt2"
path_to_save_folder = "model"
path_to_lora = os.path.join(path_to_save_folder,"lora")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if DEBUG:
    device = "cpu"
print(f"Using device: {device}")


# Load GPT-2 Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# Add a separate pad_token
tokenizer.pad_token = tokenizer.eos_token
#if tokenizer.pad_token is None:
#    # Use '<|PAD|>' as the padding token
#    tokenizer.add_special_tokens({'pad_token': '<|PAD|>'})
#    pad_token_id = tokenizer.pad_token_id
#    print("Added new pad_token '<|PAD|>' with ID:", pad_token_id)
batch_size = 16
max_length=None#128
train_dataset, val_dataset,test_dataset,train_loader,val_loader,test_loader, pad_token_id = get_enc_dataset(data_path,
                                                                                   tokenizer,
                                                                                   batch_size=batch_size,
                                                                                   max_length = max_length
                                                                                  )

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Index(['Text', 'Label'], dtype='object')
Max Length is:  65
Index(['Text', 'Label'], dtype='object')
Max Length is:  65
Index(['Text', 'Label'], dtype='object')
Max Length is:  65
Max Length:  65
Number of training batches: 3573, Number of validation batches: 149


## Load Petrained Model

### Test bevor Training

In [4]:
# %%

import torch.nn as nn
from transformers import GPT2ForSequenceClassification, GPT2Config
#tokenizer.pad_token = tokenizer.eos_token
from src.lora_helper import calc_accuracy,forward_for_classification

model_name = "gpt2"
num_labels = 2
model_config = GPT2Config.from_pretrained(model_name, num_labels=num_labels)
model = GPT2ForSequenceClassification.from_pretrained(model_name, config=model_config)
model.config.pad_token_id = tokenizer.pad_token_id 
model.to(device)
model.eval()

init_train_acc = calc_accuracy(train_loader, model, device, max_batches=10)
init_val_acc   = calc_accuracy(val_loader, model, device, max_batches=10)
#init_test_acc  = calc_accuracy(test_loader, model, device, max_batches=10)
init_test_acc = 0.0
print(f"Initial Accuracies -> Train: {init_train_acc*100:.2f}%, Val: {init_val_acc*100:.2f}%, Test: {init_test_acc*100:.2f}%")

2025-01-18 01:06:53.248448: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-18 01:06:53.266401: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737158813.289198   87324 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737158813.296247   87324 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-18 01:06:53.318846: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Initial Accuracies -> Train: 52.50%, Val: 48.12%, Test: 0.00%


## Different Strategy

### Strategy A: Full Fine-Tuning

In [ ]:
# %%
from src.lora_train import train_model_full_finetune
import copy
import time
import torch 
import torch.nn as nn
is_dry=False
print("=== Strategy A: Full Fine-Tuning ===")
modelA = copy.deepcopy(model)
full_tune_params = sum(p.numel() for p in modelA.parameters() if p.requires_grad)
print(f"[Full Fine-Tuning] Trainable Params: {full_tune_params}")


init_accA = calc_accuracy(train_loader, modelA, device, max_batches=1)
print(f"[Full Fine-Tuning] Initial Train Acc (first 10 batches): {init_accA*100:.2f}%")

#elapsedB, lora_train_losses, lora_val_accs,train_acc_l 
elapsedA, ft_train_losses, ft_val_accs, train_acc_l  = train_model_full_finetune(
    modelA, train_loader, val_loader, device, epochs=6, lr=5e-5,is_dry=is_dry
)

train_accA = calc_accuracy(train_loader, modelA, device,is_dry=is_dry)
val_accA   = calc_accuracy(val_loader, modelA, device,is_dry=is_dry)
test_accA  = calc_accuracy(test_loader, modelA, device,is_dry=is_dry)
print(f"[Full Fine-Tuning] Time: {elapsedA:.2f}s, TrainAcc={train_accA*100:.2f}%, ValAcc={val_accA*100:.2f}%, TestAcc={test_accA*100:.2f}%\n")

=== Strategy A: Full Fine-Tuning ===
[Full Fine-Tuning] Trainable Params: 124441344
[Full Fine-Tuning] Initial Train Acc (first 10 batches): 43.75%
Epoch 1/6, step 500/3573, loss = 0.1961
Epoch 1/6, step 1000/3573, loss = 0.1994
Epoch 1/6, step 1500/3573, loss = 0.2621
Epoch 1/6, step 2000/3573, loss = 0.2521
Epoch 1/6, step 2500/3573, loss = 0.5072
Epoch 1/6, step 3000/3573, loss = 0.3664
Epoch 1/6, step 3500/3573, loss = 0.2179
Epoch 1/6, step 3570/3573, loss = 0.4453Epoch=1, Loss=0.2836, ValAcc=93.12% TrainAcc=88.22%
Epoch 2/6, step 500/3573, loss = 0.2414
Epoch 2/6, step 1000/3573, loss = 0.1089
Epoch 2/6, step 1500/3573, loss = 0.0770
Epoch 2/6, step 2000/3573, loss = 0.1568
Epoch 2/6, step 2500/3573, loss = 0.1587
Epoch 2/6, step 3000/3573, loss = 0.0165
Epoch 2/6, step 3500/3573, loss = 0.1963
Epoch 2/6, step 3570/3573, loss = 0.0704Epoch=2, Loss=0.1578, ValAcc=93.08% TrainAcc=94.16%
Epoch 3/6, step 500/3573, loss = 0.0095
Epoch 3/6, step 1000/3573, loss = 0.2402
Epoch 3/6, step

In [ ]:
from src.eval_helper import * 
#path_to_partial path_to_lora
train_run_label = "temp_full"
#elapsedA, ft_train_losses, ft_val_accs, train_acc_l 
#save_everything(path_to_full, train_run_label, elapsedA, 
#                 ft_train_losses,train_acc_l, ft_val_accs,train_accA,
#                 val_accA,test_accA,modelA)

save_everything(path_to_save_folder=path_to_full,
                 train_run_label=train_run_label,
                 elapsed=elapsedA,
                 train_losses=ft_train_losses,
                 train_acc=train_acc_l,
                 val_accs=ft_val_accs,
                 train_acc_complete=train_accA,
                 val_acc_complete=val_accA,
                 test_acc_complete=test_accA,
                 model=modelA)
"""
save_everything(path_to_save_folder=,
                 train_run_label=,
                 elapsed=,
                 train_losses=,
                 train_acc=,
                 val_accs=,
                 train_acc_complete=,
                 val_acc_complete=,
                 test_acc_complete=,
                 model=)
                 """

## Strategy B LoRa

In [ ]:
# %%
from src.lora_model import *
from src.lora_train import train_model_lora
import copy
import time
is_dry=False
print("=== Strategy B: LoRA ===")
##6 Epochs
modelB = copy.deepcopy(model)  # Duplicate the base model
replace_modules_with_lora(modelB, rank=16, alpha=32)  # Replace layers
modelB.to(device)
elapsedB, lora_train_losses, lora_val_accs,train_acc_l = train_model_lora(
    modelB, train_loader, val_loader, device, epochs=6, lr=1e-4, log_grad_norms=True,is_dry=is_dry
)

def calc_accuracy_full(loader, model, device, max_batches=None,is_dry=False):
    model.eval()
    correct, total = 0, 0
    #for i, (input_ids, attention_mask, y_batch) in enumerate(loader):
    for i, instance in enumerate(train_loader):
        input_ids = instance['input_ids']
        attention_mask = instance['attention_mask']
        y_batch = instance['labels']
        #-
        if max_batches and (i+1) > max_batches:
            break
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        y_batch = y_batch.to(device)
        with torch.no_grad():
            logits = forward_for_classification(model, input_ids, attention_mask, device)
        preds = torch.argmax(logits, dim=-1)
        correct += (preds == y_batch).sum().item()
        total   += y_batch.size(0)
        if is_dry:
            break
    return correct / total if total > 0 else 0

train_accB = calc_accuracy_full(train_loader, modelB, device,is_dry=is_dry)
val_accB   = calc_accuracy_full(val_loader, modelB, device,is_dry=is_dry)
test_accB  = calc_accuracy_full(test_loader, modelB, device,is_dry=is_dry)
print(f"[LoRA] Training Time: {elapsedB:.2f}s")
print(f"[LoRA] TrainAcc={train_accB*100:.2f}%, ValAcc={val_accB*100:.2f}%, TestAcc={test_accB*100:.2f}%\n")

In [ ]:
from src.eval_helper import * 
#path_to_partial path_to_lora
train_run_label = "lora_ep6_saveUpdate2"
"""
save_everything(path_to_lora, train_run_label, elapsedB, lora_train_losses,
                lora_val_accs,
                train_accB,
                val_accB,test_accB,
                modelB
               )
"""
save_everything(path_to_save_folder=path_to_lora,
                 train_run_label=train_run_label,
                 elapsed=elapsedB,
                 train_losses=lora_train_losses,
                 train_acc=train_acc_l,
                 val_accs=lora_val_accs,
                 train_acc_complete=train_accB,
                 val_acc_complete=val_accB,
                 test_acc_complete=test_accB,
                 model=modelB)

 #### B.3: Advanced Metrics (Precision, Recall, F1)

 We'll calculate a more comprehensive set of metrics on the LoRA model to evaluate performance beyond accuracy.

In [ ]:
# %%
from src.lora_helper import advanced_metrics
accB, precB, recB, f1B = advanced_metrics(test_loader, modelB, device)
print(f"[LoRA Advanced Metrics on Test] Accuracy={accB*100:.2f}%, Precision={precB*100:.2f}%, Recall={recB*100:.2f}%, F1={f1B*100:.2f}%")